# Self-Driving Car Engineer Nanodegree
***
## Project 2: **Advanced Lane Finding** 

#### **Written by: Philip Renn**

[image]: ./output_images/writeup_images/result.png
---

![alt text][image]

---

## Preliminary Goal
    Calibrate a 2D camera using chessboard calibration images to a aquire a conversion matrix and distortion coefficients.   

## Primary Goal
    Write a sofetware pipeline to identiify the lane boundaries from raw images from a video.

### Steps:
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

***
## Import Packages

In [1]:
# Import packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
import glob
import pickle
%matplotlib inline

# Import packages to edit/save/watch video clips
from PIL import Image
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from collections import deque

***
## Initialize Variables
These variables are used in pipeline functions which are called for each raw image in the video. By initializing these variables outside of the functions, processing time is reduced without affecting the result as their values are not modified by the funtions.

In [2]:
##################################################################
###################### INITIALIZE VARIABLES ######################
# Load camera calibration file and extract cals 
pickle_dict = pickle.load(open("calibration_data.p", "rb"))
mtx = pickle_dict["mtx"]
dist = pickle_dict["dist"]

#### Perspective Transform Variables ####
# Set offset for destination points
offsetx = 400
# Defining trapezoid points for src image 
imshape = (720, 1280)
left_bottom_p = [217, 705]
left_top_p = [588, 453]
right_top_p = [694, 453]
right_bottom_p = [1100, 705]
# Set src and dest points and get matrix (and inverse) to warp perspective
# Define 4 source points src = np.float32([[,],[,],[,],[,]])
src = np.float32([[left_bottom_p, left_top_p, right_top_p, right_bottom_p]], dtype=np.int32)
# Define 4 destination points dst = np.float32([[,],[,],[,],[,]])
dst = np.float32([[offsetx, imshape[0]],[offsetx, 0],[imshape[1]-offsetx, 0],[imshape[1]-offsetx, imshape[0]]])
# Use cv2.getPerspectiveTransform() to get M, the transform matrix, and its inverse, Minv
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

#### Region of Interest Variables ####
# Set 'verticies' for ROI mask
left_bottom = [300, imshape[0]]
left_top = [0,0]
right_top = [int(imshape[1]), 0]
right_bottom = [int(imshape[1] - 300), int(imshape[0])]
vertices = np.array([[left_bottom, left_top, right_top, right_bottom]], dtype=np.int32)

#### Thesholds for Color Layers ####
# Defining color space thresholds for R, H, L, S layers
R_thresh = (220,255)  # Red:        {0-255}
H_thresh = (25,100)   # Hue:        {0-180}
L_thresh = (0,190)    # Lightness:  {0-255}
S_thresh = (100,255)  # Saturation: {0-255}
############################## (END) #############################
##################################################################

***
## Image Processing Functions
The following funtions are called to filter the lane line pixels which will be used to determine the lane boundary.

In [3]:
##################################################################
################## IMAGE PROCESING FUNCTIONS #####################
def undistort(img):
    ''' Corrects for distortion caused by camera lenses '''
    return cv2.undistort(img,mtx,dist,None,mtx)

def elevate_perspective(img):
    ''' Warps image to achieve a top-down view of road '''
    # Use cv2.warpPerspective() to warp input image to a top-down view
    img_size = (img.shape[1], img.shape[0])
    # Use 'M' imported from 'calibration_data.p' file
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped

def grayscale(img):
    """Applies the Grayscale transform"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def sobel(img):
    '''
    Performs sobel operation to get the gradient in x & y direction.
    Thresholds the magnitude direction which is achieved using x & y gradients
    Returns images for x-direction gradient and the thresholded magnitude direction. 
    '''
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take the derivative in x & y
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    # Take the absolute value of the derivative or gradient
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)    
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Create a mask of 1's where the scaled gradient magnitude is within threshold
    sobel_thresh = (30,255)
    gradx = np.zeros_like(scaled_sobelx)
    gradx[(scaled_sobelx >= sobel_thresh[0]) & (scaled_sobelx <= sobel_thresh[1])] = 1
    # Get the binary image of the thresholded magnitude direction
    dir_binary = sobel_mag_dir(abs_sobelx, abs_sobely)
    return gradx, dir_binary

def sobel_magnitude(gray, mag_thresh=(50,255)):
    ''' Retuns a binary thesholded image of gradient magnitude. '''
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=9)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=9)
    gradmag = np.hypot(sobelx, sobely)
    # Scale magnitude from 0-255
    gradmag = (gradmag / (np.max(gradmag)/255)).astype(np.uint8)
    # Create binary image of thresholded magnitude
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

def sobel_mag_dir(abs_sobelx, abs_sobely, dir_thresh=(0.0, 0.3)):
    ''' Returns thesholded binary image of magitude direction from sobel gradients '''
    # Calculate absolute direction of gradient
    absgraddir = np.arctan2(abs_sobelx, abs_sobely)
    # Create binary image of thresholded gradient direction
    binary_output = np.zeros_like(absgraddir)
    binary_output[(absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1])] = 1
    return binary_output

def bgr2hls(img):
    ''' Converts BGR image to HLS color space '''
    return cv2.cvtColor(img, cv2.COLOR_BGR2HLS) 

def color_mask(img, img_hls):
    """
    Applies a color selection mask to find yellow & white lane line pixel values.
    Converts input image to HLS color space and thresholds each layer and performs
    bitwise operations with the Red layer from BGR image.
    
    Returns a color-masked binary image.
    """
    # Grab R layer of BGR image and H,L,S layers from HLS image
    R = img[:,:,2]
    H = img_hls[:,:,0]
    L = img_hls[:,:,1]
    S = img_hls[:,:,2]
    
    # Applying color thresholds using color space thresholds for R, H, L, S layers
    R_binary = np.zeros_like(R)
    R_binary[(R >= R_thresh[0]) & (L >= 150)] = 1
    H_binary = np.zeros_like(H)
    H_binary[(H >= H_thresh[0]) & (H <= H_thresh[1])] = 1
    L_binary = np.zeros_like(L)
    L_binary[(L >= L_thresh[0]) & (L <= L_thresh[1])] = 1
    S_binary = np.zeros_like(S)
    S_binary[(S >= S_thresh[0]) & (S <= S_thresh[1])] = 1
    
    # Combine hue and Lightness layers to detect lane lines in darker areas (e.g. shadows)
    HL_binary = cv2.bitwise_and(H_binary, L_binary)

    # Combine all thresholded layers
    mask_binary = np.zeros_like(R)
    mask_binary = cv2.bitwise_and(HL_binary, S_binary)
    mask_binary = cv2.bitwise_or(mask_binary, R_binary)    
    return mask_binary

def region_of_interest(img):
    """
    Applies a regional mask on an image.
    Only keeps the region of the image defined by the trapezoid formed from
    `vertices` (see initialation block above). 
    
    Retuns masked image with the rest of the image set to black.
    """
    # Defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    # Defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    # Filling pixels inside the trapezoid defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    # Returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image 
############################## (END) #############################
##################################################################

***
## Lane Detection Functions
These functions are used to determine (and display) the lane boundaries, curvature radius and vehicle offset.

In [4]:
##################################################################
################### LANE DETECTION FUNCTIONS #####################
def find_lane_pixels(binary_warped):
    ''' This functions uses the window search method to find the lane pixels. '''
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # Find the peak of the left and right halves of the histogram
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # These will be the starting point for window to search for the left and right line
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Set number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 70
    # Set minimum number of pixels found to recenter window
    minpix = 40

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        # Set boundaries of the window
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If found(pixels) > minpix(pixels), recenter next window at mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def search_around_poly(binary_warped, left, right):
    ''' 
    This function will search around the previously detected lane lines for the next lane lines pixels.
    Requires a previously detected lane line the binary image.
    '''
    # HYPERPARAMETER
    # Set width of the margin around the previous polynomial to search
    margin = 70

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Define polynomial to search arround
    left_fit = left.current_fit
    right_fit = right.current_fit
    
    # Set the area of search based on activated x-values within +/- margin of polynomial function
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(binary_warped, left, right):
    '''
    Detect lane line pixels and fit the best fit polynomial to lane line. 
    
    This function uses sanity checks to verify that the lane line pixels return a
    polinomial that is similar to the previously detected line 
    '''
    # Find lane line pixels
    if left.detected is True and right.detected is True:  # search around polynomial if previous lines detected
        leftx, lefty, rightx, righty, out_img = search_around_poly(binary_warped, left, right)
    else:  # use sliding window if previous line not detected
        leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Generate 72 evenly spaced y-values that span the image height (less y_vals, faster processing)
    y_vals = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0]//10)

    # Fit a second order polynomial to each line using `np.polyfit`
    try: 
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
    # If error deteccted, set new fit as best_fit, and set current_fit to None to skip sanity checks
    except TypeError:
        left_fit = left.best_fit
        right_fit = right.best_fit
        left.current_fit = None
        right.current_fit = None
    
    try:
        left_fitx = left_fit[0]*y_vals**2 + left_fit[1]*y_vals + left_fit[2]
        right_fitx = right_fit[0]*y_vals**2 + right_fit[1]*y_vals + right_fit[2]

        # Sanity checks for x-intercepts and x values 
        if ((len(left.current_fit) > 1) & (len(right.current_fit) > 1)):
            # Find difference between the best 
            left.diffs = np.abs(left_fit - left.current_fit)
            right.diffs = np.abs(right_fit - right.current_fit)
            left_x_diff = np.abs(left_fitx[0] - left.bestx[0])
            right_x_diff = np.abs(right_fitx[0] - right.bestx[0])

            # Sanity checks
            # Left line - check if similar to current line
            if ((left.diffs[2] < 50) & (left_x_diff < 60)):
                left.detected = True
                left.current_fit = left_fit
                left.bestx = weighted_average(left, left_fitx)
            # New line is not similar to current, try window search for next frame 
            else:
                left.detected = False
                left.current_fit = []
                left.bestx = weighted_average(left, left_fitx)
                left.recent_xfitted.clear

            # Right Line - check if similar to current line
            if ((right.diffs[2] < 50) & (right_x_diff < 60)):
                right.detected = True
                right.current_fit = right_fit
                right.bestx = weighted_average(right, right_fitx)
            # New line is not similar to current, try window search for next frame 
            else:
                right.detected = False
                right.current_fit = []
                right.bestx = weighted_average(right, right_fitx)
                right.recent_xfitted.clear
        
        # If no current_fit, set it as the new fit (e.g. first frame, error)         
        else:
            left.detected = True
            right.detected = True
            left.current_fit = left_fit
            right.current_fit = right_fit
            # Set bestx as new x-fitted values if empty, otherwise clear queue 
            if left.bestx == None:
                left.bestx = left_fitx
            else:
                left.recent_xfitted.clear
                
            if right.bestx == None:
                right.bestx = right_fitx
            else:
                right.recent_xfitted.clear

    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left.detected = False
        right.detected = False
        left.current_fit = None
        right.current_fit = None
        # If their bestx is empty, assign a polynomial and clear the queue of previously detected x-fitted values
        if left.bestx is None:
            left.bestx = 1*y_vals**2 + 1*y_vals
            left.recent_xfitted.clear
        if right.bestx is None:
            right.bestx = 1*y_vals**2 + 1*y_vals
            right.recent_xfitted.clear

    # Calculate the best fit using the best x values  
    left.best_fit = np.polyfit(y_vals, left.bestx, 2)
    right.best_fit = np.polyfit(y_vals, right.bestx, 2)
    
    # Create image for output
    zero_output = np.dstack((binary_warped, binary_warped, binary_warped))*255
    lines_image = np.zeros_like(zero_output)
    draw_lane(lines_image, left.bestx, right.bestx, y_vals)
    return lines_image, y_vals

def weighted_average(h, fitx, n=7):
    """ 
    Applies a weighted average to last 'n' x-fitted values to find the best fit
    for lane line.
    
    h: handle of class object
    """
    weights=[1,1,2,3,5,8,13]#,21,34,55]
    run_len = len(h.recent_xfitted)  # get current length of queue

    # Maintaining running window |f0,f1,f2,...,fn| of previous n x-values
    if fitx is not None:
        if run_len == n:  # queue is full
            h.recent_xfitted.popleft()  # [f0<--,|f1,f2,...,fn|,fn+1]
        else:             # queue not full
            run_len += 1  # add 1 to length
        h.recent_xfitted.append(fitx)  # [f0,|f1,f2,...,fn<--|,fn+1]
    else:  # reduce queue if no x-fitted values are found
        if len(h.recent_xfitted) > 0:
            h.recent_xfitted.popleft()
            run_len -= 1

    # Compute weighted average
    if len(h.recent_xfitted) > 0:
        h.bestx = np.average(h.recent_xfitted, axis=0, weights=weights[:run_len])
    return h.bestx

def curvature_radius_real(y_vals, left, right):
    ''' Calculates current radius of curvature in meters '''
    # Assign metric value to pixels in each dimension
    ym_per_pix = 30/720  # meters per pixel in y dimension
    xm_per_pix = 3.7/(right.best_fit[-1] - left.best_fit[-1]) # meters per pixel in x dimension
    
    y_eval = np.max(y_vals)
    
    # Convert best fit polynomial to real world values
    left_fit_cr = np.polyfit(y_vals*ym_per_pix, left.bestx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(y_vals*ym_per_pix, right.bestx*xm_per_pix, 2)
    
    # Calculate radius of curvature
    left.radius_of_curvature = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right.radius_of_curvature = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

def vehicle_offset(img, left, right):
    ''' Calculates difference between vehicle center and lane center '''
    vehicle_center = img.shape[1] // 2  # center of vehicle is center of image
    xm_per_pix = 3.7 / (right.bestx[-1] - left.bestx[-1])  # x-values of line nearest the vehicle
    # Calculate x-value of lane center
    left.line_base_pos = left.bestx[-1] - vehicle_center    # expects negative value
    right.line_base_pos = right.bestx[-1] - vehicle_center  #expects positive value
    vehicle_offset = left.line_base_pos + right.line_base_pos
    return vehicle_offset*xm_per_pix

def draw_lane(img, left_fitx, right_fitx, y_vals):
    ''' Draws lane on input 'img' '''
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, y_vals]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, y_vals])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(img, np.int_([pts]), (0,255,0))

def original_perspective(dest, warped_image, Minv):
    ''' Unwarps the top-down view bck to the original perspective using Minv '''
    img_size = (dest.shape[1], dest.shape[0])
    original_img = cv2.warpPerspective(warped_image, Minv, img_size, flags=cv2.INTER_LINEAR)
    return original_img

def weighted_img(init_img, img, α=1., β=0.3, γ=0.):
    """
    `img` is a blank image with the lane drawn on it from the original perspective.
    
    `init_img` is original unditorted image before any processing.
    
    The result image is computed as follows:
        initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape
    """
    return cv2.addWeighted(init_img, α, img, β, γ)

def detected_lane(init_img, img, lane_offset, left, right):
    ''' 
    This function will write the curvature radius and vehicle offset on input image.
    The curvature radius is calculated by averaging the radius of each line.
    '''
    # Average of left and right curvature radii
    radius_curvature = np.int32((left.radius_of_curvature + right.radius_of_curvature) / 2)
    # Write curvature radius value
    cv2.putText(init_img, "Radius of Curvature: %s m" % str(radius_curvature), (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    # Determine relative offset from lane center to vehicle center 
    if lane_offset < 0:  # left of center
        offset_dir = str('{:.2f}'.format(np.abs(lane_offset)) + "m left of center")
    elif lane_offset > 0:  # right of center
        offset_dir = str('{:.2f}'.format(lane_offset) + "m right of center")
    else:
        offset_dir = "center"
    # Write vehicle offset
    cv2.putText(init_img, "Vehicle is %s" % offset_dir, (20, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    return weighted_img(init_img, img)

############################## (END) #############################
##################################################################

***
## Lane Finding Pipeline

In [5]:
##################################################################
#################### LANE DETECTION PIPELINE #####################
# Define lane-finding pipeline function
def process_image(img):
    """
    This function serves as a pipeline to process each frame of the video file
    using the functions defined above in a sequential method designed to undistort
    the input image, find each lane line, determine the lane position, and calculate
    curvature radius and vehicle offset.    
    Returns a manipulated image of the input image with the lane region highlighted and info.
    """  
    #### COPY OF IMAGE ####
    image = np.copy(img)
    
    #### UNDISTORT IMAGE ####
    undist = undistort(image)
    
    #### PERSPECTIVE TRANSFORM ####
    image_T = elevate_perspective(undist)   
    
    #### COLOR SELECTION ####
    hls_image_T = bgr2hls(image_T)
    color_select = color_mask(image_T, hls_image_T)
    
    #### SOBEL & MAGNITUDE DIRECTION ####
    gradx, dir_binary = sobel(image_T)
    
    #### COMBINE COLOR & SOBEL RESULTS ####
    color_gradx = cv2.bitwise_or(color_select, gradx)
    filtered_image = np.zeros_like(color_select)
    filtered_image[(color_gradx == 1) & (dir_binary == 0)] = 1
    
    #### REGION OF INTEREST ####
    masked_image = region_of_interest(filtered_image) # applies trapezoidal ROI
        
    #### FIND LANE PIXELS ####
    output_image, y_vals = fit_polynomial(masked_image, Left, Right)
        
    #### CALCULATE REAL-WORLD RADIUS OF CURVATURE ####
    curvature_radius_real(y_vals, Left, Right)

    #### FIND OFFSET OF VEHICLE CENTER TO LANE CENTER #### 
    lane_offset = vehicle_offset(output_image, Left, Right)
        
    #### TRANSFORM IMAGE BACK TO ORIGINAL PERSPECTIVE #### 
    result_image = original_perspective(undist, output_image, Minv)

    #### DISPLAY DETECTED LANE, CURVATURE RADIUS, VEHICLE OFFSET ON ORIGINAL UNDISTORTED IMAGE ####
    lane_image = detected_lane(undist, result_image, lane_offset, Left, Right)

    return lane_image #masked_img
############################## (END) #############################
##################################################################

***
## Line() Class

In [6]:
##################### CREATE 'Line()' CLASS ######################
class Line:
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = deque() 
        #average x values of the fitted line over the last n iterations
        self.bestx = None    
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = np.array([])  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None
        #y values for detected line pixels
        self.ally = None  


***
## Image Processing
The code in this section will process all images in the 'test_images' folder and display the result. Press 'Q' to close each image window.

In [7]:
test_images = glob.glob('test_images/*.jpg')

for fimage in test_images:
    Left = Line()
    Right = Line()
    image = cv2.imread(fimage)
    result = process_image(image)
    
    cv2.imshow('Result', result)
    r = cv2.waitKey(0) & 0xFF
    if r == ord('q'):
        cv2.destroyWindow('Result')

cv2.destroyAllWindows()

***
## Video Processing
The code in this section will process each frame of the 'project_video.mp4' and save the result video to 'project_video_output.mp4'.

### project_video.mp4

In [8]:
##################### PROCESS VIDEO ######################
Left = Line()   # create instance of Line() class for left lane line
Right = Line()  # create instance of Line() class for right lane line
video = VideoFileClip("project_video.mp4")

# Loop through and process each frame of video file
processed_video = video.fl_image(process_image)

%time processed_video.write_videofile("project_video_output.mp4", audio=False)

video.close()

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [04:30<00:00,  4.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

Wall time: 4min 31s


In [9]:
# Display output video post-processing
project_video_output = "project_video_output.mp4"
HTML("""
<video width="960" height="540" controls> 
  <source src="{0}">
</video>
""".format(project_video_output))